In [1]:
import tensorflow as tf
from utils.distributions import Gaussian, gen_ring, GMM
from utils.dynamics import Dynamics

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [100]:
def tf_accept(x, Lx, px):
    mask = (px - tf.random_uniform(tf.shape(px)) >= 0.)
    return tf.where(mask, Lx, x)

gaussian1 = Gaussian(mu=np.zeros((2,)), sigma=np.eye(2,))
gaussian2 = Gaussian(mu=np.ones((2,)), sigma=np.array([[1.0, 0.95], [0.95, 1.0]]))
n_samples = 1000
init_energy = gaussian1.get_energy_function()
final_energy = gaussian2.get_energy_function()
beta_diff = 0.0001

def body(z, w, t):
    curr_beta = (t) * beta_diff
    curr_energy = lambda z: (1-curr_beta) * init_energy(z) + (curr_beta) * final_energy(z)
    diff_energy = - final_energy(z) + init_energy(z)
    w = w + beta_diff * diff_energy
    dynamics = Dynamics(x_dim=2, energy_function=curr_energy, eps=0.5, hmc=True, T=25)
    
    new_z, _, p = dynamics.forward(z)
    mh_z = tf_accept(z, new_z, p)
    
    return mh_z, w, t+1

def cond(z, w, t):
    return tf.less(t, 10000+1)

start_z, start_w, start_t = tf.random_uniform((n_samples, 2)), tf.zeros((n_samples,)), tf.constant(0.)

last_z, last_w, T = tf.while_loop(body=body, cond=cond, loop_vars=(tf.random_normal((20, 2)), tf.zeros((20,)), tf.constant(0.)))

In [101]:
with tf.Session() as sess:
    log_w_T = sess.run(last_w)

In [17]:
def get_log_Z(sigma):
    n = sigma.shape[0]
    return 0.5 * np.log(np.linalg.det(2 * np.pi * sigma))
    

In [24]:
Z_1 = get_log_Z(np.eye(2,))

In [25]:
Z_T = get_log_Z(np.array([[1.0, 0.95], [0.95, 1.0]]))

In [78]:
from scipy.misc import logsumexp

In [67]:
  beta = tf.linspace(0., 1., n_samples+1)[1:]
  beta_diff = beta[1] - beta[0]
  def body(a, beta):
    def log_prob_beta(x, beta, *args, **kwargs):
      return (1-beta) * log_q_f(x, *args, **kwargs) + beta * log_prob_f(x, *args, **kwargs)
    last_x = a[1]
    w = a[2]
    w = w + beta_diff * (log_prob_f(last_x, *args, **kwargs)
                         - log_q_f(last_x, *args, **kwargs))
    alpha, updated_x = hmc_kernel(step_size, lf_steps, last_x, log_prob_beta,
                                  beta, *args, **kwargs)
    alpha.set_shape(batch_size)
    updated_x.set_shape(z_shape)
    return (alpha, updated_x, w)

  alpha, x, w = tf.scan(body, beta, (tf.zeros_like(initial_x[:, 0]),
                        initial_x, tf.zeros_like(initial_x[:, 0])))
  return alpha[-1], x[-1], w[-1]

-1.1639514504891684

In [116]:
logsumexp(log_w_T) - np.log(20)

1.165473018194056

In [150]:
S1 = np.random.randn(5, 5)
S2 = np.random.randn(5, 5)

mu1 = np.random.randn(5)
mu2 = np.random.randn(5)

sigma1 = S1.dot(S1.T) + 0.1 * np.eye(5)
sigma2 = S2.dot(S2.T) + 0.1 * np.eye(5)

gaussian1 = Gaussian(mu=np.zeros(5,), sigma=np.eye(5))
gaussian2 = Gaussian(mu=mu2, sigma=sigma2)

init_energy = gaussian1.get_energy_function()
final_energy = gaussian2.get_energy_function()

beta = tf.linspace(0., 1., 500+1)[1:]
beta_diff = beta[1] - beta[0]

def body(a, beta):
    curr_energy = lambda z: (1-beta) * init_energy(z) + (beta) * final_energy(z)
    last_x = a[1]
    w = a[2]
    w = w + beta_diff * (- final_energy(last_x) + init_energy(last_x))
    dynamics = Dynamics(x_dim=5, energy_function=curr_energy, eps=0.5, hmc=True, T=25)
    Lx, _, px = dynamics.forward(last_x)
    updated_x = tf_accept(last_x, Lx, px)
    return (px, updated_x, w)

initial_x = tf.random_normal((100, 5))

alpha, x, w = tf.scan(body, beta, (tf.zeros_like(initial_x[:, 0]),
                    initial_x, tf.zeros_like(initial_x[:, 0])))

In [151]:
with tf.Session() as sess:
    log_w_T = sess.run(w[-1])

In [152]:
get_log_Z(np.eye(5)) - get_log_Z(sigma2)

-3.0705821639835591

In [154]:
logsumexp(log_w_T) - np.log(100)

2.9927192373822695